In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'co2': 0.0008}
band = [4]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3b
  atmpro = trp
  band = 3b
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.55, 1.5, 1.66)]
  wgt = [(0.8, 0.8, 0.8, 0.6, 0.6, 0.9)]
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)]
  wgt = [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]


# Cooling Rate by Layer

CRD             CLIRAD `coolr_bands`           
            layer      coolrg                layer     coolrg
pressure                                                     
0.000312        1  115.307350                    1  74.431304
0.000750        2   49.922455                    2  56.701649
0.001052        3   39.583668                    3  48.889597
0.001476        4   31.229220                    4  39.890072
0.002070        5   24.655664                    5  30.385440
0.002904        6   19.625550                    6  21.717281
0.004074        7   15.802071                    7  15.077690
0.005714        8   12.863105                    8  11.066165
0.008015        9   10.539571                    9   9.084352
0.011243       10    8.617240                   10   8.096396
0.015771       11    6.943956                   11   7.216290
0.022122       12    5.419996                   12   6.078958
0.031031       13    3.951255                   13   4.591614
0.043528       14    2.201424                   14   2.515367
0.061057       15    0.886534                   15   0.757670
0.085645       16    1.381950                   16   0.872037
0.120136       17    2.700289                   17   2.036666
0.168516       18    3.647248                   18   3.059081
0.236378       19    4.584580                   19   4.328593
0.331549       20    5.665576                   20   5.900100
0.465100       21    7.007790                   21   7.568954
0.652400       22    9.065155                   22   9.459557
0.915100       23   11.408312                   23  11.140423
1.283650       24   10.948356                   24  10.293634
1.800600       25    8.443308                   25   8.319891
2.525700       26    6.587892                   26   7.041189
3.542800       27    5.293868                   27   5.647776
4.969550       28    4.307406                   28   4.184671
6.970850       29    3.553431                   29   3.466988
9.778100       30    2.929484                   30   3.278745
13.715850      31    2.309271                   31   2.810373
19.239350      32    1.676063                   32   1.876742
26.987250      33    1.196560                   33   1.154001
37.855300      34    0.876549                   34   0.947211
53.100050      35    0.557002                   35   0.720926
73.887500      36    0.260287                   36   0.307482
97.662500      37    0.127412                   37   0.064889
121.437500     38    0.107536                   38   0.040625
145.212500     39    0.016217                   39  -0.026926
168.987500     40   -0.122661                   40  -0.151413
192.762500     41   -0.112788                   41  -0.124520
216.537500     42    0.019713                   42   0.020977
240.312500     43    0.070244                   43   0.068994
264.087500     44    0.082355                   44   0.078749
287.862500     45    0.085197                   45   0.083414
311.637500     46    0.083845                   46   0.082386
335.412500     47    0.080551                   47   0.080909
359.187500     48    0.076453                   48   0.075496
382.962500     49    0.072308                   49   0.069382
406.737500     50    0.068437                   50   0.063172
430.512500     51    0.064431                   51   0.057032
454.287500     52    0.059608                   52   0.049235
478.062500     53    0.053773                   53   0.042409
501.837500     54    0.047593                   54   0.034910
525.612500     55    0.041920                   55   0.028774
549.387500     56    0.037145                   56   0.023970
573.162500     57    0.033191                   57   0.020352
596.937500     58    0.029935                   58   0.017679
620.712500     59    0.027075                   59   0.015632
644.487500     60    0.024490                   60   0.014016
668.262500     61    0.022204                   61   0.012939
692.037500     62    0.020265                 

# Fluxes by Layer

CRD                           CLIRAD             \
                        flug       fldg      fnetg       flug       fldg   
pressure    level                                                          
0.000000    1     -17.833660   0.000000 -17.833660 -17.737596   0.000001   
0.000624    2     -17.834090   0.008958 -17.825132 -17.738096   0.006006   
0.000876    3     -17.834296   0.010652 -17.823644 -17.738305   0.007905   
0.001229    4     -17.834556   0.012566 -17.821990 -17.738577   0.010220   
0.001723    5     -17.834880   0.014720 -17.820160 -17.738926   0.012907   
0.002417    6     -17.835285   0.017152 -17.818133 -17.739376   0.015855   
0.003391    7     -17.835796   0.019926 -17.815870 -17.739959   0.018943   
0.004757    8     -17.836448   0.023134 -17.813314 -17.740722   0.022145   
0.006672    9     -17.837290   0.026895 -17.810395 -17.741723   0.025657   
0.009359    10    -17.838396   0.031355 -17.807040 -17.743054   0.029881   
0.013128    11    -17.839873   0.036680 -17.803193 -17.744846   0.035287   
0.018415    12    -17.841893   0.043049 -17.798845 -17.747309   0.042270   
0.025830    13    -17.844738   0.050655 -17.794083 -17.750783   0.051084   
0.036232    14    -17.848930   0.059716 -17.789215 -17.755857   0.061816   
0.050823    15    -17.855961   0.070551 -17.785410 -17.764073   0.074381   
0.071291    16    -17.868244   0.084983 -17.783260 -17.777771   0.089916   
0.100000    17    -17.886573   0.108012 -17.778561 -17.797742   0.112853   
0.140271    18    -17.910601   0.144922 -17.765679 -17.823664   0.148492   
0.196760    19    -17.941483   0.200209 -17.741273 -17.856587   0.201886   
0.275997    20    -17.980316   0.282074 -17.698242 -17.897296   0.283229   
0.387100    21    -18.026984   0.403306 -17.623677 -17.945045   0.408637   
0.543100    22    -18.077292   0.583113 -17.494178 -17.994891   0.598367   
0.761700    23    -18.108065   0.848627 -17.259439 -18.023803   0.872256   
1.068500    24    -18.060777   1.215946 -16.844830 -17.976066   1.229434   
1.498800    25    -17.916530   1.629760 -16.286770 -17.833500   1.611610   
2.102400    26    -17.718526   2.035459 -15.683067 -17.637308   2.010357   
2.949000    27    -17.481442   2.459047 -15.022395 -17.402209   2.481463   
4.136600    28    -17.207725   2.930069 -14.277655 -17.127750   3.001614   
5.802500    29    -16.900873   3.473232 -13.427642 -16.815842   3.515584   
8.139200    30    -16.556741   4.112685 -12.444057 -16.457845   4.117344   
11.417000   31    -16.164126   4.857526 -11.306600 -16.045153   4.977852   
16.014700   32    -15.732294   5.683393 -10.048901 -15.587668   6.051139   
22.464000   33    -15.319712   6.551265  -8.768447 -15.143855   7.041241   
31.510500   34    -14.956809   7.470623  -7.486186 -14.745425   7.879593   
44.200100   35    -14.595525   8.426943  -6.168582 -14.344805   8.902942   
62.000000   36    -14.265234   9.271106  -4.994128 -13.975901  10.054287   
85.775000   37    -14.144066   9.882988  -4.261079 -13.824250  10.768691   
109.550000  38    -14.218947  10.316701  -3.902246 -13.877471  11.004681   
133.325000  39    -14.348408  10.749017  -3.599391 -13.972524  11.214160   
157.100000  40    -14.658559  11.104840  -3.553719 -14.245642  11.411439   
180.875000  41    -15.424661  11.525491  -3.899171 -15.012817  11.752143   
204.650000  42    -16.577360  12.360544  -4.216816 -16.204247  12.592849   
228.425000  43    -17.793700  13.632402  -4.161297 -17.463372  13.911059   
252.200000  44    -18.991499  15.028032  -3.963467 -18.697033  15.339048   
275.975000  45    -20.165470  16.433942  -3.731529 -19.902263  16.766083   
299.750000  46    -21.315528  17.823942  -3.491586 -21.078873  18.177637   
323.525000  47    -22.441339  19.185886  -3.255453 -22.229248  19.560061   
347.300000  48    -23.542450  20.513853  -3.028597 -23.351091  20.909792   
371.075000  49    -24.618060  21.804780  -2.813280 -24.445424  22.216768   
394.850000  50    -25.666270  23.056632  -2.609638 -25.510277  23.477041 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')